# Data Preprocessing

Handling missing values, cleaning data, merging datasets for geolocation analysis, feature engineering.

# Dataset Overview

**Fraud_Data.csv**
Includes e-commerce transaction data aimed at identifying fraudulent activities.
- `user_id`: A unique identifier for the user who made the transaction.
- `signup_time`: The timestamp when the user signed up.
- `purchase_time`: The timestamp when the purchase was made.
- `purchase_value`: The value of the purchase in dollars.
- `device_id`: A unique identifier for the device used to make the transaction.
- `source`: The source through which the user came to the site (e.g., SEO, Ads).
- `browser`: The browser used to make the transaction (e.g., Chrome, Safari).
- `sex`: The gender of the user (M for male, F for female).
- `age`: The age of the user.
- `ip_address`: The IP address from which the transaction was made.
- `class`: The target variable where 1 indicates a fraudulent transaction and 0 indicates a non-fraudulent transaction.

**IpAddress_to_Country.csv**

Maps IP addresses to countries

- `lower_bound_ip_address`: The lower bound of the IP address range.
- `upper_bound_ip_address`: The upper bound of the IP address range.
- `country`: The country corresponding to the IP address range.

In [1]:
# Import necessary libraries

import pandas as pd

import socket
import struct


import os, sys
# Add the 'src' directory to the Python path for module imports
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

# Configure logging
from logger import SetupLogger
# Assuming this class is defined in src/
from data_preprocessing import LoadData  

logger = SetupLogger(log_file='../logs/preprocessing.log').get_logger()

### Loading the Dataset

In [2]:
# Initialize the DataPreprocessor with the logger and the path to the dataset
data_pre = LoadData(filepath='../data/raw/Fraud_Data.csv', logger=logger)

In [3]:
fraud_df = data_pre.load_dataset()

In [4]:
# First few rows of the dataset
fraud_df.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


In [5]:
# Dataset Information
fraud_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         151112 non-null  int64  
 1   signup_time     151112 non-null  object 
 2   purchase_time   151112 non-null  object 
 3   purchase_value  151112 non-null  int64  
 4   device_id       151112 non-null  object 
 5   source          151112 non-null  object 
 6   browser         151112 non-null  object 
 7   sex             151112 non-null  object 
 8   age             151112 non-null  int64  
 9   ip_address      151112 non-null  float64
 10  class           151112 non-null  int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 12.7+ MB


### Check for missing values in the dataset

In [6]:
missing_values = fraud_df.isnull().sum()
print("Missing Values in Each Column:")
missing_values

Missing Values in Each Column:


user_id           0
signup_time       0
purchase_time     0
purchase_value    0
device_id         0
source            0
browser           0
sex               0
age               0
ip_address        0
class             0
dtype: int64

The `signup_time` and `purchase_time` will be converted to standard datetime formatas following code:

In [7]:
fraud_df['signup_time'] = pd.to_datetime(fraud_df['signup_time'], errors='coerce')
fraud_df['purchase_time'] = pd.to_datetime(fraud_df['purchase_time'], errors='coerce')

In [8]:
fraud_df.dtypes

user_id                    int64
signup_time       datetime64[ns]
purchase_time     datetime64[ns]
purchase_value             int64
device_id                 object
source                    object
browser                   object
sex                       object
age                        int64
ip_address               float64
class                      int64
dtype: object

### Check for duplicate rows in the DataFrame

In [9]:
duplicate_count = fraud_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

if duplicate_count:
    fraud_df.drop_duplicates(inplace=True)

Number of duplicate rows: 0


### understand the statistical summary of the dataset as follows

In [10]:
fraud_df.describe()

,user_id,signup_time,purchase_time,purchase_value,age,ip_address,class
count,151112.000000,151112,151112,151112.000000,151112.000000,1.511120e+05,151112.000000
mean,200171.040970,2015-04-20 00:56:09.511329280,2015-06-16 02:56:38.759952896,36.935372,33.140704,2.152145e+09,0.093646
min,2.000000,2015-01-01 00:00:42,2015-01-01 00:00:44,9.000000,18.000000,5.209350e+04,0.000000
25%,100642.500000,2015-02-18 09:52:48.500000,2015-04-18 14:41:25.500000,22.000000,27.000000,1.085934e+09,0.000000
50%,199958.000000,2015-04-19 04:41:30,2015-06-18 13:46:17.500000,35.000000,33.000000,2.154770e+09,0.000000
75%,300054.000000,2015-06-18 14:47:22.750000128,2015-08-17 18:48:31.500000,49.000000,39.000000,3.243258e+09,0.000000
max,400000.000000,2015-08-18 04:40:29,2015-12-16 02:56:05,154.000000,76.000000,4.294850e+09,1.000000
std,115369.285024,NaN,NaN,18.322762,8.617733,1.248497e+09,0.291336


### Key insights from the summary statistics

**Data Overview**: The dataset contains 151,112 records with the following features:

- `user_id`: Ranges from 2 to 400,000, with a mean of 200,171.04.

- `purchase_value`: Ranges from 9 to 154, with a mean of 36.94 and a standard deviation of 18.32.

- `age`: Ranges from 18 to 76, with a mean of 33.14 and a standard deviation of 8.62.

- `ip_address`: Values range significantly, indicating a wide geographical distribution.

- `class`: Binary variable with a mean of 0.09, suggesting a low prevalence of the positive class.

## Geolocation Analysis

Merge datasets (Fraud and IP) for geolocation analysis.

In [11]:
def ip_to_int(ip):
    """Convert an IP address to its integer representation."""
    try:
        return struct.unpack("!I", socket.inet_aton(ip))[0]
    except socket.error:
        return None  # Handle invalid IPs gracefully

# Load the datasets
fraud_data = fraud_df.copy()
ip_country_data = pd.read_csv('../data/raw/IpAddress_to_Country.csv')

# Convert IP addresses in fraud data to integer format
fraud_data['ip_int'] = fraud_data['ip_address'].apply(lambda x: ip_to_int(str(int(x))) if not pd.isna(x) else None)

# Drop rows with invalid IPs
fraud_data.dropna(subset=['ip_int'], inplace=True)

# Convert bounds in the country data to integer
ip_country_data['lower_bound_ip_address'] = ip_country_data['lower_bound_ip_address'].astype(int)
ip_country_data['upper_bound_ip_address'] = ip_country_data['upper_bound_ip_address'].astype(int)

# Sort both datasets for merge_asof
fraud_data.sort_values('ip_int', inplace=True)
ip_country_data.sort_values('lower_bound_ip_address', inplace=True)

# Merge the datasets using merge_asof
merged_data = pd.merge_asof(
    fraud_data,
    ip_country_data,
    left_on='ip_int',
    right_on='lower_bound_ip_address',
    direction='backward'
)

# Filter rows where ip_int is within the lower and upper bounds
merged_data = merged_data[(merged_data['ip_int'] >= merged_data['lower_bound_ip_address']) &
                          (merged_data['ip_int'] <= merged_data['upper_bound_ip_address'])]

# Drop unnecessary columns
merged_data.drop(columns=['lower_bound_ip_address', 'upper_bound_ip_address'], inplace=True)

# Save to csv file
merged_data.to_csv('../data/processed/merged_fraud_data.csv')
# Display the first few rows of the merged dataset with country information
merged_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,ip_int,country
634,247547,2015-06-28 03:00:34,2015-08-09 03:57:29,47,KIXYSVCHIPQBR,SEO,Safari,F,30,1.677886e+07,0,16778864,Australia
635,220737,2015-01-28 14:21:11,2015-02-11 20:28:28,15,PKYOWQKWGJNJI,SEO,Chrome,F,34,1.684205e+07,0,16842045,Thailand
636,390400,2015-03-19 20:49:09,2015-04-11 23:41:23,44,LVCSXLISZHVUO,Ads,IE,M,29,1.684366e+07,0,16843656,China
637,69592,2015-02-24 06:11:57,2015-05-23 16:40:14,55,UHAUHNXXUADJE,Direct,Chrome,F,30,1.693873e+07,0,16938732,China
638,174987,2015-07-07 12:58:11,2015-11-03 04:04:30,51,XPGPMOHIDRMGE,SEO,Chrome,F,37,1.697198e+07,0,16971984,Thailand


### Load the World GeoDataFrame

Load the world shapefile containing geometries and country names after downloading a complete shapefile from [here](https://www.naturalearthdata.com/downloads/):

In [12]:
import geopandas as gpd
from geolocation_analyzer import GeolocationAnalyzer

# Load your data as DataFrames
fraud_data = pd.read_csv('../data/processed/merged_fraud_data.csv')
world_data = gpd.read_file('../data/external/110m_cultural/ne_110m_admin_0_countries.shp')

# Initialize the class with DataFrames and the logger
geo_analyzer = GeolocationAnalyzer(fraud_df=fraud_data, 
                                   world_gdf=world_data, 
                                   logger=logger)

### Distribution of Transactions by Country

In [13]:
# Calculate total transaction volumes by country
highestTranscation = geo_analyzer.calculate_transaction_volume()

highestTranscation.sort_values(by='transaction_volume', ascending=False).head(10)

,country,transaction_volume
0,United States,58049
1,China,12038
2,Japan,7306
3,United Kingdom,4490
4,Korea Republic of,4162
5,Germany,3646
6,France,3161
7,Canada,2975
8,Brazil,2961
9,Italy,1944


From the above distribution we can conclude that:

- The United States has a significantly higher transaction volume compared to other countries, indicating a robust e-commerce market.

- China and Japan also show substantial transaction volumes, suggesting strong consumer engagement in these markets.

- Countries like the United Kingdom, South Korea, and Germany contribute notable transaction volumes, highlighting a diverse global e-commerce landscape.

### Fraud Rate by Country

In [14]:
# Calculate total and fraudulent transactions by country
fraude_rate = geo_analyzer.calculate_fraud_rate()
fraude_rate.sort_values(by='fraud_rate', ascending=False).head(10)

,country,fraud_rate
166,Guadeloupe,1.000000
115,Albania,0.434783
154,Saint Kitts and Nevis,0.419355
98,Malta,0.388889
177,Gambia,0.333333
51,Philippines,0.264151
164,Cape Verde,0.262712
131,Mongolia,0.260504
20,Poland,0.245283
89,Sudan,0.233333


Madagascar has an alarming 100% fraud rate, indicating all recorded transactions are fraudulent, which may signal critical vulnerabilities in transaction security or monitoring.

Countries like Albania (43.48%), Gibraltar (41.94%), and Trinidad and Tobago (38.89%) exhibit high fraud rates, suggesting systemic issues that require urgent attention and improved security measures.

Nations such as the Philippines (26.42%), Vanuatu (26.27%), and Afghanistan (26.05%) show substantial fraud rates, highlighting the necessity for strengthened digital security infrastructure and proactive fraud detection strategies.

## Feature Engineering

Features extracted from Fraud_Data.csv

- Normalization and Scaling

- Encode Categorical Features

- Transaction frequency and velocity

- Device transaction frequency

- Time-Based features

In [15]:
# Import the class
from data_preprocessing import FeatureEngineering

feature_engineering = FeatureEngineering(fraud_df, logger)
feature_engineering.pipeline()
processed_data = feature_engineering.get_processed_data()
processed_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,age,ip_address,class,hour_of_day,day_of_week,...,user_transaction_frequency,device_transaction_frequency,user_transaction_velocity,source_Direct,source_SEO,browser_FireFox,browser_IE,browser_Opera,browser_Safari,sex_M
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,-0.160204,QVPSPJUOCKZAR,0.679914,7.327584e+08,0,-1.377455,0.991020,...,0.0,-0.261514,-0.230128,False,True,False,False,False,False,True
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,-1.142592,EOGFQPIZPYXFZ,2.304476,3.503114e+08,0,-1.522122,-1.501259,...,0.0,-0.261514,-0.229874,False,False,False,False,False,False,False
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,-1.197169,YSSKYOSJHPPLJ,2.304476,2.621474e+09,1,0.937208,-0.005891,...,0.0,3.941861,4.345476,False,True,False,False,True,False,True
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,0.385567,ATGTXKYKUDUQN,0.911994,3.840542e+09,0,0.213876,-1.501259,...,0.0,-0.261514,-0.230120,False,True,False,False,False,True,True
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,0.112681,NAUITBZFJKHWW,1.376155,4.155831e+08,0,0.937208,-0.504347,...,0.0,-0.261514,-0.230128,False,False,False,False,False,True,True


In [16]:
# Save the processed datase to the csv for modeling
processed_data.to_csv('../data/processed/processed_fraud_data.csv')